In [ ]:
import datajoint as dj
dj.config["enable_python_native_blobs"] = True
dj.config['schema_name'] = "nnfabrik_toy_V4"
schema = dj.schema("nnfabrik_toy_V4")

import os
from os import listdir
from os.path import isfile, join


import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle 
import time
from torch import load

import nnfabrik
from nnfabrik import main, builder
from nnfabrik.main import *

import nnvision
import seaborn as sns

In [28]:
from nnvision.utility.measures import get_explainable_var
n_repeats = np.hstack([np.arange(2,10), np.arange(10,20,2),  np.arange(20,36,4)])
n_iters = 10

v4_hash= '293317b0600244345fe605961ef9f170'
v1_hash = '839b51124b143e456cfcf1d78954ea7d'

In [ ]:
key = dict(dataset_hash=v4_hash)
dataloaders = (Dataset() & key).get_dataloader(seed=1000)

rep_array, score_array=[],[]
for i in range(n_iters):
    for reps in n_repeats:
        score = get_explainable_var(dataloaders["test"], per_neuron=False, repeat_limit=reps)
        rep_array.append(reps)
        score_array.append(score)

v4_reps = rep_array
v4_scores = score_array


key = dict(dataset_hash=v1_hash)
dataloaders = (Dataset() & key).get_dataloader(seed=1000)
rep_array, score_array=[],[]
for i in range(n_iters):
    for reps in n_repeats:
        score = get_explainable_var(dataloaders["test"], per_neuron=False, repeat_limit=reps)
        rep_array.append(reps)
        score_array.append(score)

v1_reps = rep_array
v1_scores = score_array



In [5]:
df = pd.DataFrame({'repeats': np.hstack([v1_reps, v4_reps]), 
                        'explainable variance': np.hstack([v1_scores, v4_scores]),
                        'dataset': ['v1']*len(v1_reps) + ['v4']*len(v4_reps)})

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,6))
sns.set_context('talk', font_scale=0.8)

flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
plt.grid("on", ls='--')
sns.lineplot(x="repeats", y="explainable variance", hue="dataset",data=df, ci="sd", palette=flatui[-2:])
plt.ylim([0.29, 0.34])
sns.despine(trim=True, offset=2)
plt.title("Explainable Variance across repeats")
plt.savefig("Monkey_V2_explainable_variance.pdf", dpi=100, bbox_inches="tight")